# V13코드를 기반으로 lightGBM 을 사용한 버전
# V17코드를 기반으로 lightGBM 을 사용한 다른 버전

# 시각화

In [53]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # setting seaborn default for plots

def bar_chart(feature):
    yes = train[train['voted']==1][feature].value_counts()
    no = train[train['voted']==2][feature].value_counts()
    df = pd.DataFrame([yes,no])
    df.index = ['Yes','No']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [54]:
import pandas as pd

train=pd.read_csv('./train.csv')
test=pd.read_csv('./test_x.csv')

# 데이터 전처리

In [55]:
import numpy as np
family_drop_score=10

train['familysize']=np.where(train['familysize'] >= family_drop_score, family_drop_score, train['familysize'])
test['familysize']=np.where(test['familysize'] >= family_drop_score, family_drop_score, test['familysize'])

train['education']=np.where(train['education'] == 0, 2, train['education'])
test['education']=np.where(test['education'] == 0, 2, test['education'])

train['urban']=train['urban'].astype('str')
test['urban']=test['urban'].astype('str')

train['education']=train['education'].astype('str')
test['education']=test['education'].astype('str')


# 원핫 인코딩 (gender, race, religion)

In [56]:
train=pd.get_dummies(train, columns=['race'])
train=pd.get_dummies(train, columns=['religion'])
train=pd.get_dummies(train, columns=['urban'])
train=pd.get_dummies(train, columns=['hand'])
train=pd.get_dummies(train, columns=['gender'])

test=pd.get_dummies(test, columns=['race'])
test=pd.get_dummies(test, columns=['religion'])
test=pd.get_dummies(test, columns=['urban'])
test=pd.get_dummies(test, columns=['hand'])
test=pd.get_dummies(test, columns=['gender'])

In [57]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
train['age_group_code'] = label.fit_transform(train['age_group'])
train['education_code'] = label.fit_transform(train['education'])

test['age_group_code'] = label.fit_transform(test['age_group'])
test['education_code'] = label.fit_transform(test['education'])

# Feature : Mach_score (마키아벨리니즘 스코어)

In [58]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

In [59]:
reverse = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for r in reverse: 
        train[r] = 6 - train[r]
        test[r] = 6 - test[r]

In [60]:
reverse = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for r in reverse: 
        train[r] = 6 - train[r]
        test[r] = 6 - test[r]

In [61]:
train['Mach_score'] = train[Answers].mean(axis = 1)
test['Mach_score'] = test[Answers].mean(axis = 1)

# TIPI
## 성실성 (Conscientiousness)
## 우호성 (Agreeableness)
## 신경성 (Neuroticism) - 정서적 안정감
## 개방성 (Openness to experience)
## 외향성 (Extroversion)

- 성실성 : {3번 점수 + (8 - '8번 점수')} ÷ 2
- 우호성 : {7번 점수 + (8 - '2번 점수')} ÷ 2
- 정서적 안정성(점수가 낮으면 신경성과 관련) : {9번 점수 + (8 - '4번 점수')} ÷ 2
- 개방성 : {5번 점수 + (8 - '10번 점수')} ÷ 2
- 외향성 : {1번 점수 + (8 - '6번 점수')} ÷ 2

In [62]:
train['TIPT_C']=(train['tp03']+(8-train['tp08']))/2
train['TIPT_A']=(train['tp07']+(8-train['tp02']))/2
train['TIPT_N']=(train['tp09']+(8-train['tp04']))/2
train['TIPT_O']=(train['tp05']+(8-train['tp10']))/2
train['TIPT_E']=(train['tp01']+(8-train['tp06']))/2

test['TIPT_C']=(test['tp03']+(8-test['tp08']))/2
test['TIPT_A']=(test['tp07']+(8-test['tp02']))/2
test['TIPT_N']=(test['tp09']+(8-test['tp04']))/2
test['TIPT_O']=(test['tp05']+(8-test['tp10']))/2
test['TIPT_E']=(test['tp01']+(8-test['tp06']))/2

# wf_(1~3) : 설문자의 어휘 능력 (허구인 단어의 정의를 앎)

In [63]:
wf=['wf_01', 'wf_02', 'wf_03']

train['wf']=train[wf].mean(axis=1)
test['wf']=test[wf].mean(axis=1)

# drop 할 feature들

In [64]:
drop_feature=[
#                 'engnat',
    
#                 'familysize',
    
#                 'married',
    
#                 'race_Arab', 'race_Asian','race_Black','race_Indigenous Australian','race_Native American','race_Other','race_White',               
    
#                 'religion_Agnostic','religion_Atheist', 'religion_Buddhist','religion_Christian_Catholic','religion_Christian_Mormon', 
#                 'religion_Christian_Other', 'religion_Christian_Protestant', 'religion_Hindu','religion_Jewish','religion_Muslim', 
#                 'religion_Other','religion_Sikh',
    
#                 'gender_Female','gender_Male',
                
#                 'urban_0','urban_1','urban_2','urban_3',
    
                'education',
                
                'age_group',
                
                'QaA', 'QbA', 'QcA', 'QdA', 'QeA','QfA', 'QgA', 'QhA', 'QiA', 'QjA', 'QkA', 'QlA', 'QmA', 'QnA', 'QoA','QpA', 
                'QqA', 'QrA', 'QsA', 'QtA',
                
                'QaE', 'QbE', 'QcE', 'QdE', 'QeE', 'QfE', 'QgE', 'QhE', 'QiE', 'QjE', 'QkE', 'QlE','QmE', 'QnE', 'QoE', 'QpE', 
                'QqE', 'QrE', 'QsE', 'QtE',
    
#                 'p_time' , 

#                 'n_time',
              
                'tp01', 'tp02', 'tp03', 'tp04', 'tp05','tp06','tp07', 'tp08', 'tp09', 'tp10',
              
#                 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09','wr_10','wr_11', 'wr_12', 'wr_13', 
                'wr_02',  'wr_04', 'wr_07', 'wr_08', 'wr_10', 'wr_12', 'wr_13', 
                'wf_01', 'wf_02', 'wf_03', 'wf',
              
                'hand_0','hand_1','hand_2','hand_3',
              
              ]

train = train.drop(drop_feature, axis = 1)
test = test.drop(drop_feature, axis = 1)

# Model 생성

In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import lightgbm as lgb

# 사용한 feature 목록

In [66]:
for i in train:
    print(i)


index
engnat
familysize
married
voted
wr_01
wr_03
wr_05
wr_06
wr_09
wr_11
race_Arab
race_Asian
race_Black
race_Indigenous Australian
race_Native American
race_Other
race_White
religion_Agnostic
religion_Atheist
religion_Buddhist
religion_Christian_Catholic
religion_Christian_Mormon
religion_Christian_Other
religion_Christian_Protestant
religion_Hindu
religion_Jewish
religion_Muslim
religion_Other
religion_Sikh
urban_0
urban_1
urban_2
urban_3
gender_Female
gender_Male
age_group_code
education_code
Mach_score
TIPT_C
TIPT_A
TIPT_N
TIPT_O
TIPT_E


# 모델 전처리

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from lightgbm import LGBMClassifier

import warnings
import gc
warnings.filterwarnings("ignore")

X = train.copy()
X.drop('voted', axis=1, inplace = True)
Y = train['voted']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [72]:
import lightgbm as lgb

model=LGBMClassifier(n_estimators=400)
evals=[(x_test, y_test)]
model.fit(x_train, y_train, early_stopping_rounds=100, eval_metric="AUC",eval_set=evals, verbose=True)

[1]	valid_0's auc: 0.758352	valid_0's binary_logloss: 0.668553
[2]	valid_0's auc: 0.759454	valid_0's binary_logloss: 0.651548
[3]	valid_0's auc: 0.759731	valid_0's binary_logloss: 0.637381
[4]	valid_0's auc: 0.760145	valid_0's binary_logloss: 0.625689
[5]	valid_0's auc: 0.761448	valid_0's binary_logloss: 0.615765
[6]	valid_0's auc: 0.761788	valid_0's binary_logloss: 0.607471
[7]	valid_0's auc: 0.763108	valid_0's binary_logloss: 0.600341
[8]	valid_0's auc: 0.763672	valid_0's binary_logloss: 0.594339
[9]	valid_0's auc: 0.763734	valid_0's binary_logloss: 0.58925
[10]	valid_0's auc: 0.764497	valid_0's binary_logloss: 0.5847
[11]	valid_0's auc: 0.765235	valid_0's binary_logloss: 0.58096
[12]	valid_0's auc: 0.765984	valid_0's binary_logloss: 0.577502
[13]	valid_0's auc: 0.766252	valid_0's binary_logloss: 0.574548
[14]	valid_0's auc: 0.766567	valid_0's binary_logloss: 0.572126
[15]	valid_0's auc: 0.766779	valid_0's binary_logloss: 0.570066
[16]	valid_0's auc: 0.766877	valid_0's binary_logloss

LGBMClassifier(n_estimators=400)

In [95]:
preds=model.predict(x_test)

In [96]:
pred_proba=model.predict_proba(test)

In [97]:
pred_proba.shape

(11383, 2)

In [98]:
submission= pd.read_csv('./sample_submission.csv')
submission['voted'] = pred_proba[:,1]
submission.to_csv('submission_proba.csv', index = False)

In [99]:
submission.head()

,index,voted
0,0,0.591548
1,1,0.903608
2,2,0.481670
3,3,0.181750
4,4,0.800976
